# osiris

![img](https://dm2301files.storage.live.com/y4mmRC1xelS6Y6MEqUnZ-k2vjpADHpo6UMZAaZWROunr9-Ml5FYDlZ6WMxCGedy7NDhwDpusZdF5E1oLR5Qn6momydHe7tYUOMwNeFeGW7pUWkBjGPSnZp2sacYWs9IKkose6xjhSySL_v2tbfItRI7T_Pw_Tayhaa2F_vrwW6ucyr6WPa6s9DWH_if9Y5Y3yAU?width=375&height=250&cropmode=none)


osiris is a Python data processing and analysis environment for data-based computational conflict forecasting using very large datasets and graph-based methods and models and visualization, powered by scalable graph databases.

You can use osiris to analyze causal chains and networks of confict and violence around the world from realtime-updated, [automatically-encoded political event data](https://parusanalytics.com/eventdata/papers.dir/Schrodt_Yonamine_NewDirectionsInText.pdf) from projects like GDELT. This notebook gives an overview of the osiris project, the [GDELT project](https://www.gdeltproject.org/) data that osiris uses, how to import political event data using osiris either from the GDELT file server or from Google BigQuery, how to visualize and analyze it using Python, and how to load it into a TigerGraph graph server instance to efficiently run graph-centric queries on it to retrieve vertex-edge event data that can then be further analyzed.

## Notebook Environment Setup

In [1]:
import os, sys
# Check if running inside Colab or Kaggle
IN_COLAB = 'COLAB_GPU' in os.environ
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IN_HOSTED_NB = IN_COLAB or IN_KAGGLE
os.environ['IN_HOSTED_NB'] = str(IN_HOSTED_NB)

OS_NAME = sys.platform.upper()
if OS_NAME in ['LINUX', 'DARWIN'] and IN_HOSTED_NB:
  import subprocess
  print('Installing osiris from GitHub...')
  print(subprocess.run('if [ -d "osiris" ]; then rm -Rf osiris; fi', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('git clone https://github.com/allisterb/osiris --recurse-submodule', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('cd osiris && ./install', text=True, shell=True, check=True, capture_output=True).stdout)

# If we're not in a hosted nb env assume we're running Jupyter from the osiris project directory root
OSIRIS_PATH = '..' if not IN_HOSTED_NB else 'osiris'

# Import the osiris code and set the runtime env. 
sys.path.append(os.path.join(OSIRIS_PATH, 'osiris'))
sys.path.append(os.path.join(OSIRIS_PATH, 'ext'))
from osiris_global import set_runtime_env
set_runtime_env(interactive_nb=True)

## GDELT Event Data

*From the  [GDELT project](https://www.gdeltproject.org/) website*:
>The GDELT Project is a realtime network diagram and database of global human society for open research.
![gf](https://www.gdeltproject.org/images/spinningglobe.gif)

>The GDELT Project is an initiative to construct a catalog of human societal-scale behavior and beliefs across all countries of the world, connecting every person, organization, location, count, theme, news source, and event across the planet into a single massive network that captures what's happening around the world, what its context is and who's involved, and how the world is feeling about it, every single day.

The GDELT [event data](http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf) contains hundreds of millions of automatically coded events extracted from news stories daily using NLU methods and models. Each event data row contains the following fields:
1. *Actors*: Humans or organizations or states which initiate and are the target of event actions. Actors may have geographic information but not temporal. An event references exactly 2 actors: Actor1 and Actor2.
2. *Actions*: Codes and other information which describe each event. Actions have both temporal and spatial attributes: an event time plus some geo information like latitude / longitude.  
3. *SourceURL*: a URL that locates the *story* from which the event data was extracted.

osiris can extract data directly from the GDELT file server. The advantage of this method is that you don't need to have any special credentials or server access (remember we're interested *open-source* indicators.). All the data is downloaded directly to your client machine or notebook environment.

### Importing GDELT data from file server

osiris uses *DataSource* classes to manage importing tabular data. 

In [2]:
# Import data directly from GDELT file server
from data.gdelt import DataSource
import pandas as pd
gdelt = DataSource()

In [3]:
# Get event data for a 1 week period
events = gdelt.import_data('events', 'Apr-20-2022', 'Apr-20-2022')

Importing GDELT events data for 1 day(s) from 04-20-2022 to 04-20-2022...


Import GDELT events data:   0%|          | 0/1 [00:00<?, ?day/s]

Importing GDELT events data for 1 day(s) from 04-20-2022 to 04-20-2022 completed in 11.53 s.


About a week's worth of event data in 2022 consists of about 700K events takes up about 340MB RAM.

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125670 entries, 0 to 125669
Data columns (total 62 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          125670 non-null  int64  
 1   SQLDATE                125670 non-null  int64  
 2   MonthYear              125670 non-null  int64  
 3   Year                   125670 non-null  int64  
 4   FractionDate           125670 non-null  float64
 5   Actor1Code             113887 non-null  object 
 6   Actor1Name             113887 non-null  object 
 7   Actor1CountryCode      72667 non-null   object 
 8   Actor1KnownGroupCode   1744 non-null    object 
 9   Actor1EthnicCode       572 non-null     object 
 10  Actor1Religion1Code    1641 non-null    object 
 11  Actor1Religion2Code    391 non-null     object 
 12  Actor1Type1Code        53316 non-null   object 
 13  Actor1Type2Code        3732 non-null    object 
 14  Actor1Type3Code        96 non-null  

In [5]:
events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1040205582,20210420,202104,2021,2021.3014,CRM,KIDNAPPER,NaN,NaN,NaN,...,3,"Harrison County, Mississippi, United States",US,USMS,NaN,30.4091,-89.04360,695747,20220420013000,https://ussanews.com/2022/04/19/grand-jury-won...
1,1040205583,20210420,202104,2021,2021.3014,CRM,KIDNAPPER,NaN,NaN,NaN,...,3,"Harrison County, Mississippi, United States",US,USMS,NaN,30.4091,-89.04360,695747,20220420013000,https://ussanews.com/2022/04/19/grand-jury-won...
2,1040205584,20210420,202104,2021,2021.3014,CRM,KIDNAPPER,NaN,NaN,NaN,...,3,"Harrison County, Mississippi, United States",US,USMS,NaN,30.4091,-89.04360,695747,20220420013000,https://ussanews.com/2022/04/19/grand-jury-won...
3,1040205585,20210420,202104,2021,2021.3014,LEG,SENATOR,NaN,NaN,NaN,...,3,"Lancaster County, Nebraska, United States",US,USNE,NaN,40.7833,-96.68360,835876,20220420013000,https://siouxcityjournal.com/opinion/other-voi...
4,1040205586,20210420,202104,2021,2021.3014,RUS,RUSSIA,RUS,NaN,NaN,...,4,"Bengaluru, Karnataka, India",IN,IN19,70159,12.9833,77.58330,-2090174,20220420013000,https://kelo.com/2022/04/19/ibm-forecasts-upbe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125665,1040379005,20220420,202204,2022,2022.3014,idgLEG,INDIGENOUS,NaN,NaN,idg,...,4,"Red River, Alberta, Canada",CA,CA01,12536,58.4000,-114.75000,-568101,20220420230000,https://www.cbc.ca/news/politics/manitoba-meti...
125666,1040379006,20220420,202204,2022,2022.3014,kisGOV,KISII,NaN,NaN,kis,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420230000,https://www.standardmedia.co.ke/politics/artic...
125667,1040379007,20220420,202204,2022,2022.3014,nav,NAVAJO,NaN,NaN,nav,...,3,"White House, District of Columbia, United States",US,USDC,NaN,38.8951,-77.03640,531871,20220420230000,https://sofrep.com/news/the-navajo-code-talker...
125668,1040379008,20220420,202204,2022,2022.3014,sco,SCOTTISH,NaN,NaN,sco,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420230000,https://www.eastlothiancourier.com/news/200820...


Event data is highly denormalized with many redundancies for ease of querying and coded using a hierachical coding system called [CAMEO](http://data.gdeltproject.org/documentation/CAMEO.Manual.1.1b3.pdf) - Conflict and Mediation Event Observations

In [6]:
events[['EventCode', 'CAMEOCodeDescription']]

,EventCode,CAMEOCodeDescription
0,120,"Reject, not specified below"
1,173,"Arrest, detain, or charge with legal action"
2,193,Fight with small arms and light weapons
3,010,"Make statement, not specified below"
4,010,"Make statement, not specified below"
...,...,...
125665,040,"Consult, not specified below"
125666,051,Praise or endorse
125667,042,Make a visit
125668,010,"Make statement, not specified below"


We can query and filter event data directly using the Pandas dataframe

In [7]:
# Find all events that were geolocated in Ukraine
uka_events = events[(events.ActionGeo_CountryCode == 'UP')]
uka_events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
16,1040205598,20220413,202204,2022,2022.2822,IGOUNO,UNITED NATIONS,NaN,UNO,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220420013000,https://www.irishtimes.com/news/ireland/irish-...
106,1040205688,20220420,202204,2022,2022.3014,NaN,NaN,NaN,NaN,NaN,...,4,"Lviv, L'vivs'ka Oblast', Ukraine",UP,UP15,28555,49.8407,24.0305,-1045268,20220420013000,https://www.winnipegfreepress.com/local/from-o...
107,1040205689,20220420,202204,2022,2022.3014,NaN,NaN,NaN,NaN,NaN,...,4,"Odesa, Odes'ka Oblast, Ukraine",UP,UP17,28558,46.4639,30.7386,-1049092,20220420013000,https://www.winnipegfreepress.com/local/from-o...
124,1040205706,20220420,202204,2022,2022.3014,NaN,NaN,NaN,NaN,NaN,...,4,"Odesa, Odes'ka Oblast, Ukraine",UP,UP17,28558,46.4639,30.7386,-1049092,20220420013000,https://www.winnipegfreepress.com/local/from-o...
127,1040205709,20220420,202204,2022,2022.3014,NaN,NaN,NaN,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420013000,http://www.ecns.cn/news/2022-04-20/detail-ihax...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125594,1040378934,20220420,202204,2022,2022.3014,USAGOV,THE WHITE HOUSE,USA,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220420230000,https://www.loudountimes.com/opinion/ukraine-t...
125634,1040378974,20220420,202204,2022,2022.3014,USAMED,PEN AMERICA,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420230000,https://www.arabnews.com/node/2067386/media
125635,1040378975,20220420,202204,2022,2022.3014,USAMED,PEN AMERICA,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420230000,https://www.arabnews.com/node/2067386/media
125636,1040378976,20220420,202204,2022,2022.3014,USAMED,PEN AMERICA,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420230000,https://www.arabnews.com/node/2067386/media


So about 50K of 700K events last week were coded as happening in Ukraine, not surprising given recent events. Many of those related to use of military force.

In [8]:
# CAMEO code 190 denotes 'use of military force'
uka_events[uka_events.EventCode.str.startswith('190')]

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
965,1040206547,20220420,202204,2022,2022.3014,RUS,RUSSIA,RUS,NaN,NaN,...,4,"Donbas, Ukraine (general), Ukraine",UP,UP00,25090,48.5000,38.5000,-1038077,20220420013000,https://kstp.com/kstp-news/top-news/live-updat...
1057,1040206639,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Zlatopol, Ukraine (general), Ukraine",UP,UP00,28552,48.8219,31.6539,-1060929,20220420013000,https://www.itv.com/news/2022-04-19/in-putins-...
1066,1040206648,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Dniepr River, Ukraine (general), Ukraine",UP,UP00,28557,46.4975,32.3086,-2902197,20220420013000,https://www.itv.com/news/2022-04-19/in-putins-...
1067,1040206649,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Dniepr River, Ukraine (general), Ukraine",UP,UP00,28557,46.4975,32.3086,-2902197,20220420013000,https://www.itv.com/news/2022-04-19/in-putins-...
1073,1040206655,20220420,202204,2022,2022.3014,UKRGOVMIL,UKRAINE,UKR,NaN,NaN,...,4,"Dniepr River, Ukraine (general), Ukraine",UP,UP00,28557,46.4975,32.3086,-2902197,20220420013000,https://www.itv.com/news/2022-04-19/in-putins-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120004,1040373135,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
120005,1040373136,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
120086,1040373217,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Chernihiv, Chernihivs'ka Oblast', Ukraine",UP,UP02,28554,51.5055,31.2849,-1037057,20220420221500,http://www.msn.com/en-us/news/world/a-bomb-sni...
120161,1040373292,20220420,202204,2022,2022.3014,USA,UNITED STATES,USA,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220420221500,http://www.msn.com/en-us/news/world/as-a-new-u...


In [9]:
# Import Folium to plot these military force events on a map
import folium
folium.Map(
    location=[48., 31.], 
    tiles="Stamen Toner",
    zoom_start=6
)

In [24]:
uka_map = folium.Map(
    location=[48., 31.], 
    #tiles="Stamen Toner",
    zoom_start=6
)
uka_map
uka_events_sample = uka_events[uka_events.EventCode.str.startswith('190')].sample(n=100)
for r in uka_events_sample.itertuples():
    m = folium.Marker(location=[r.ActionGeo_Lat, r.ActionGeo_Long],
                      icon=folium.Icon(color="red", icon="fire", prefix="glyphicon"),
                      tooltip=str(r.Actor1CountryCode) + '->' + str(r.EventCode) + ' ' +  str(r.CAMEOCodeDescription) + '->' + str(r.Actor2CountryCode) +' on ' + str(r.SQLDATE)
                     )
    m.add_to(uka_map)
uka_map

### Shaping tabular data into graph nodes and vertices

In [26]:
from data.etl import shape_events_vertices
events_vertices, actors_vertices = shape_events_vertices(uka_events_sample)
events_vertices

Hashing Actor1 ID:   0%|          | 0/100 [00:00<?, ?row/s]

Hashing Actor2 ID:   0%|          | 0/100 [00:00<?, ?row/s]

Creating Action ADM Code:   0%|          | 0/100 [00:00<?, ?row/s]

,ID,Actor1ID,Actor2ID,Date,IsRoot,MonthYear,Year,FractionDate,EventCode,CAMEOCodeDescription,...,Geo_ADMCode,AvgTone,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
43362,1040255400,KWMeiqCpVFcd8W3cJWAFpul7D7s=,jjyTV/XkrwQ2Pgjl5nuy4JjnmhM=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP07 25036,-5.059524,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220420090000,https://omaha.com/news/article_784d6fc3-d424-5...
55390,1040275412,QqPkBOYtke2M27FJBcZUL5NmOx4=,6M9ZcNCz9uopuy9ZgtMuz44jenA=,2022-04-20,True,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP08 28557,-3.586498,4,Kherson; Khersons'ka Oblast'; Ukraine,UP,46.6558,32.6178,-1041356,20220420111500,https://menafn.com/1104053088/Russian-army-goe...
71497,1040303135,CBW/3J723uMOFpBP3ytpWpcJl+U=,V3QsApTKRUt1qveHAeIWRLciZ8Q=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP13 28554,-3.729768,4,Irpin; Kyyivs'ka Oblast'; Ukraine,UP,50.5218,30.2506,-1040233,20220420141500,https://www.timesofisrael.com/liveblog-april-2...
57349,1040278320,DITNI6CWVozl3tLRNVzu7XdxZWY=,tlifxqsNyCzxIJnRwtQKuZToQQw=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP00 28547,-3.194103,4,Azovstal; Ukraine (general); Ukraine,UP,47.0833,37.5833,-1034725,20220420114500,https://www.washingtonexaminer.com/news/facing...
32671,1040235977,TFxqEP2MKIPkNbNZaE21O3Y3x7k=,3OAfKp0DXOluhCwyT+qp1FCx8Lo=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP00 28557,-2.122241,4,Dnipro; Ukraine (general); Ukraine,UP,46.4975,32.3086,-2902197,20220420061500,https://www.battlecreekenquirer.com/story/news...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24847,1040231064,F9WbGUBdlTjHrPaDIFNCKL4s8IU=,hN8q7CJ2s0qPE3F9dLJMRGWcov0=,2022-04-20,True,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP,-4.829545,1,Ukraine,UP,49.0000,32.0000,UP,20220420053000,HTTP://en.ce.cn/main/latest/202204/20/t2022042...
12336,1040193347,KWMeiqCpVFcd8W3cJWAFpul7D7s=,JhiE2zUf/ijgUEvh5QBdzdQNZhc=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP07 25036,-4.015320,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220420000000,https://www.recorderonline.com/news/national_n...
50835,1040267438,Qa1ILO1Ma1ZTIX7xKrRHT5saqiA=,HntCuLiLvOoIJwKofg8qvPpLp34=,2022-04-20,True,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP00 25090,-3.400000,4,Donbas; Ukraine (general); Ukraine,UP,48.5000,38.5000,-1038077,20220420103000,https://www.ucanews.com/news/ukrainian-marine-...
43229,1040255267,WZGazLRDm0gctQcGnX4mlpXfKHI=,tlifxqsNyCzxIJnRwtQKuZToQQw=,2022-04-20,False,202204,2022,2022.3014,190,"Use conventional military force, not specifie...",...,UP12 28554,-6.382002,4,Kyiv; Kyyiv; Misto; Ukraine,UP,50.4333,30.5167,-1044367,20220420090000,https://www.dailymail.co.uk/news/article-10734...


In [12]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 48672 to 4343
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     100 non-null    int64         
 1   Actor1ID               100 non-null    object        
 2   Actor2ID               100 non-null    object        
 3   Date                   100 non-null    datetime64[ns]
 4   IsRoot                 100 non-null    bool          
 5   MonthYear              100 non-null    int64         
 6   Year                   100 non-null    int64         
 7   FractionDate           100 non-null    float64       
 8   EventCode              100 non-null    object        
 9   CAMEOCodeDescription   100 non-null    object        
 10  EventBaseCode          100 non-null    object        
 11  EventRootCode          100 non-null    object        
 12  QuadClass              100 non-null    int64         
 13  

In [13]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 48672 to 4343
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Actor1Code             0 non-null      float64
 1   Actor1CountryCode      0 non-null      float64
 2   Actor1EthnicCode       0 non-null      float64
 3   Actor1Geo_ADM1Code     0 non-null      float64
 4   Actor1Geo_ADM2Code     0 non-null      float64
 5   Actor1Geo_CountryCode  0 non-null      float64
 6   Actor1Geo_FeatureID    0 non-null      float64
 7   Actor1Geo_FullName     0 non-null      float64
 8   Actor1Geo_Lat          0 non-null      float64
 9   Actor1Geo_Long         0 non-null      float64
 10  Actor1Geo_Type         0 non-null      float64
 11  Actor1KnownGroupCode   0 non-null      float64
 12  Actor1Name             0 non-null      float64
 13  Actor1Religion1Code    0 non-null      float64
 14  Actor1Religion2Code    0 non-null      float64
 15  A

In [23]:
actors_ = pd.DataFrame()
actors_['ActorID'] = actors.ActorID
actors_['ActorCode'] = actors.Actor1Code
actors_['ActorName'] = actors.Actor1Name
actors_['ActorCountryCode'] = actors.Actor1CountryCode
actors_['ActorKnownGroupCode'] = actors.Actor1KnownGroupCode
actors_['ActorEthnicCode'] = actors.Actor1EthnicCode
actors_['ActorReligion'] = actors.Actor1Religion
actors_['ActorType'] = actors.Actor1Type
actors_.info()

AttributeError: 'DataFrame' object has no attribute 'Actor1Religion'

In [ ]:
# Uncomment and run below if running inside Colab and you want to pull env variables from a file called vars.env on your GDrive
# !pip install colab-env --upgrade
# import colab_env